# Projet SUP’TRADING
#### Récupération des dernières valeurs du CAC40 et envoie automatique d'e-mails

Imports

In [1]:
import csv
import pandas as pd
import yfinance as yf
import datetime
import time
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

C:\Users\badi01\AppData\Local\Temp\ipykernel_42288\1819596784.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Etape 1 : Récupération du dernier fichier CAC 40 et export sous fichier excel

In [2]:
# Définition de la plage de dates et la liste des actions CAC40, les trois dernières années
start_date = datetime.datetime.now() - datetime.timedelta(days=365*3)
end_date = datetime.datetime.now()

cac40_stocks = [
    'AC.PA',    # Accor
    'AI.PA',    # Air Liquide
    'AIR.PA',   # Airbus
    'ALO.PA',   # Alstom
    'MT.PA',    # ArcelorMittal
    'CS.PA',    # AXA
    'BN.PA',    # Danone
    'EN.PA',    # Bouygues
    'CAP.PA',   # Capgemini
    'CA.PA',    # Carrefour
    'ACA.PA',   # Crédit Agricole
    'BNP.PA',   # BNP Paribas
    'ENGI.PA',  # ENGIE
    'EL.PA',    # EssilorLuxottica
    'RMS.PA',   # Hermès
    'OR.PA',    # L'Oréal
    'LR.PA',    # Legrand
    'MC.PA',    # LVMH
    'ML.PA',    # Michelin
    'ORA.PA',   # Orange
    'RI.PA',    # Pernod Ricard
    'UG.PA',    # Peugeot
    'PUB.PA',   # Publicis Groupe
    'RNO.PA',   # Renault
    'SAF.PA',   # Safran
    'SGO.PA',   # Saint-Gobain
    'SAN.PA',   # Sanofi
    'SU.PA',    # Schneider Electric
    'GLE.PA',   # Société Générale
    'SW.PA',    # Sodexo
    'STM.PA',   # STMicroelectronics
    'HO.PA',    # Thales
    'FP.PA',    # TotalEnergies
    'ATO.PA',   # Atos
    'VIE.PA',   # Veolia
    'DG.PA',    # Vinci
    'VIV.PA',   # Vivendi
    'WLN.PA',   # Worldline
    'KER.PA',   # Kering
    'FR.PA'     # Valeo
    ]

In [14]:
def download_stock_data(stock, start_date, end_date):
    """
    Télécharge les données historiques d'une action depuis Yahoo Finance pour un symbole d'action donné et une plage de dates donnée.

    :param stock: Le symbole d'action (ticker) pour lequel télécharger les données, par exemple 'AAPL' pour Apple Inc.
    :type stock: str
    :param start_date: La date de début de la plage de données historiques, en tant qu'objet datetime.
    :type start_date: datetime.datetime
    :param end_date: La date de fin de la plage de données historiques, en tant qu'objet datetime.
    :type end_date: datetime.datetime
    :return: Un DataFrame pandas contenant les données historiques de l'action spécifiée et de la plage de dates.
                Le DataFrame a les colonnes suivantes : 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock'.
    :rtype: pandas.DataFrame
    """
    # Téléchargement les données historiques de l'action spécifiée et de la plage de dates
    stock_data = yf.download(stock, start=start_date, end=end_date)

    # Réinitialisation de l'index du DataFrame pour faire de 'Date' une colonne au lieu de l'index
    stock_data.reset_index(inplace=True)

    # Ajout d'une colonne 'Stock' au DataFrame et définit sa valeur sur le symbole de l'action
    stock_data['Stock'] = stock

    
    return stock_data


In [15]:

# Fonction pour agréger les données des actions
def aggregate_stocks(stocks, output_path, start_date, end_date):
    """
    Télécharge les données historiques des actions d'une liste de symboles d'actions et enregistre les données dans un fichier CSV.

    :param stocks: Une liste de symboles d'actions (tickers) pour lesquels télécharger les données historiques.
    :type stocks: list of str
    :param output_path: Le chemin du fichier où les données agrégées des actions seront enregistrées sous forme de fichier CSV.
    :type output_path: str
    :param start_date: La date de début de la plage de données historiques, en tant qu'objet datetime.
    :type start_date: datetime.datetime
    :param end_date: La date de fin de la plage de données historiques, en tant qu'objet datetime.
    :type end_date: datetime.datetime
    :return: Un DataFrame pandas contenant les données historiques des actions spécifiées et de la plage de dates.
                Le DataFrame a les colonnes suivantes : 'Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Stock'.
    :rtype: pandas.DataFrame
    """
    # Initialisation d'une liste vide pour stocker les DataFrames de chaque action
    all_stock_data = []

    # Téléchargement des données des actions pour chaque symbole d'action de la liste
    for stock in stocks:
        stock_data = download_stock_data(stock, start_date, end_date)
        all_stock_data.append(stock_data)

    # Concatènation des DataFrames de chaque action en un seul DataFrame
    all_stock_data_df = pd.concat(all_stock_data, ignore_index=True)

    # Enregistrement des données agrégées des actions dans un fichier CSV
    all_stock_data_df.to_csv(output_path, index=False)
    
    return all_stock_data_df

In [16]:
# Agrègation des données des actions CAC40 et enregistrement des données dans un fichier CSV
all_stock_data_df = aggregate_stocks(
    stocks=cac40_stocks, 
    output_path='cac40_stock_data.csv',
    start_date=start_date,
    end_date=end_date
)

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AC.PA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AI.PA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AIR.PA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com'

KeyboardInterrupt: 

Failed to get ticker 'UG.PA' reason: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))


In [11]:
'''while True:
    # Obtient la date et l'heure actuelles
    current_datetime = datetime.datetime.now()

    # Vérifie s'il est temps de mettre à jour les données (toutes les 5 minutes)
    excel_cac40 = 'C:/Users/badi01/Downloads/cac40_stock_data.xls'  # Define the variable excel_cac40

    if current_datetime.minute % 5 == 0:
        # Télécharge et agrège les données des actions
        all_stock_data_df = aggregate_stocks(
            stocks=cac40_stocks,
            output_path=excel_cac40,  # Replace excel_file_path with excel_cac40
            start_date=start_date,
            end_date=end_date
        )

    # Attend 1 minute avant de vérifier à nouveau
    time.sleep(60)'''

"while True:\n    # Obtient la date et l'heure actuelles\n    current_datetime = datetime.datetime.now()\n\n    # Vérifie s'il est temps de mettre à jour les données (toutes les 5 minutes)\n    excel_cac40 = 'C:/Users/badi01/Downloads/cac40_stock_data.xls'  # Define the variable excel_cac40\n\n    if current_datetime.minute % 5 == 0:\n        # Télécharge et agrège les données des actions\n        all_stock_data_df = aggregate_stocks(\n            stocks=cac40_stocks,\n            output_path=excel_cac40,  # Replace excel_file_path with excel_cac40\n            start_date=start_date,\n            end_date=end_date\n        )\n\n    # Attend 1 minute avant de vérifier à nouveau\n    time.sleep(60)"

In [12]:
# Définition du chemin du fichier Excel
excel_cac40 = "cac40_stock_data.xlsx"

In [ ]:
# Enregistrement des données agrégées des actions dans un fichier Excel
all_stock_data_df.to_excel(excel_cac40, index=False)

### Etape 2 : Envoie de l'e-mail

In [13]:
# Boucle principale pour le suivi en temps réel
while True:
    # Obtient la date et l'heure actuelles
    current_datetime = datetime.datetime.now()

    # Vérifie s'il est temps de mettre à jour les données (toutes les heures)
    if current_datetime.hour % 1 == 0:
        # Télécharge et agrège les données des actions
        all_stock_data_df = aggregate_stocks(
            stocks=cac40_stocks,
            output_path=excel_cac40,
            start_date=start_date,
            end_date=end_date
        )

    # Vérifie si le marché boursier est fermé (17h30 heure de Paris)
    if current_datetime.hour == 17 and current_datetime.minute == 30:
        # Créez un message
        msg = MIMEMultipart()

        # Ajoutez du contenu en texte brut
        text = "voici le nouveau fichier excel."
        msg.attach(MIMEText(text, 'plain'))

        # Adresse e-mail de l'expéditeur
        from_email = "nkounkougessy7@gmail.com"

        # Mot de passe de l'expéditeur
        from_password = "skal hdvj dydx mvyh"

        # Adresse e-mail du destinataire
        recipient_email = "g_nkounkou@yahoo.com"

        # Ajoutez les adresses e-mail de l'expéditeur et du destinataire au message
        msg['From'] = from_email
        msg['To'] = recipient_email

        # Ajoutez un sujet au message
        msg['Subject'] = "FICHIER CAC 40"

        # Chemin du fichier Excel à joindre
        file_path = "C:/Users/User/cac40_stock_data.xls"

        # Créez un objet MIMEBase pour le fichier Excel
        part = MIMEBase('application', "octet-stream")

        # Ouvrez le fichier en mode binaire
        with open(file_path, 'rb') as file:
            part.set_payload(file.read())

        # Encodez le fichier en base64 pour le joindre à l'e-mail
        encoders.encode_base64(part)

        # Ajoutez un en-tête pour le fichier Excel
        part.add_header('Content-Disposition', 'attachment', filename=file_path)  

        # Ajoutez le fichier Excel à l'e-mail
        msg.attach(part)

        # Connexion au serveur SMTP de Gmail
        server = smtplib.SMTP('smtp.gmail.com', 587)

        # Commencez la connexion TLS pour la sécurité
        server.starttls()

        # Connectez-vous à votre compte Gmail
        server.login(from_email, from_password)

        # Envoie l'e-mail
        server.send_message(msg)

        # Fermeture de la connexion au serveur
        server.quit()
    # Attend 1 heure avant de vérifier à nouveau
    time.sleep(3600)

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AC.PA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AI.PA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v1/test/getcrumb (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))"))
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AIR.PA']: SSLError(MaxRetryError("HTTPSConnectionPool(host='query1.finance.yahoo.com'